# Exploring CESM output

## full, high res output data:
- for the crtl run there are differently sized files:
    - 19GB 024-199 (many files before 100-01 are not available)
    - 21GB 200-226
    - 30GB 227-268
- for the rcp run, all files are 30 GB
- in all ctrl files grid data is stored, dimensions nlat/nlon are natural numbers, but coords U/T_LONG/LAT are in degrees, depths stored in centimeter
- also all other grid data (e.g. TAREA [cm^2]) is stored in files
- grid data is also stored in binary files

## interpolated, rect output data
- grid: u/t_lon/lat store position in degrees, depth_t/w_dep store deth in meters
- no further geometric data (such as TAREA or so)

In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
# plt.style.use('dark_background')
sns.set()
sns.set_palette("tab10")
sns.set_context("notebook", font_scale=1)

In [ ]:
from paths import path_CESM
from paths import path_ocn_ctrl, path_ocn_ctrl_rect
from paths import path_ocn_rcp,  path_ocn_rcp_rect

In [ ]:
from read_binary import read_binary_2D

In [ ]:
from grid import generate_lats_lons, shift_field

# high resolution files

In [ ]:
# nc_file = f'{path_CESM}/rcp8.5_co2_f05_t12/OUTPUT/ocn/hist/daily/rcp8.5_co2_f05_t12.pop.hm.2021-01-22.nc'
# xdata   = xr.open_dataset(nc_file, decode_times=False)

In [ ]:
nc_file = f'{path_ocn_rcp}/rcp8.5_co2_f05_t12.pop.h.2000-01.nc'
xdata = xr.open_dataset(nc_file, decode_times=False)

In [ ]:
xdata

In [ ]:
xdata.coords['TLAT'][:,:]

In [ ]:
f, ax = plt.subplots(1,3,figsize=(15,5), sharey=True)
ax[0].imshow(xdata.TEMP[0,0,:,:], cmap='viridis')
ax[1].imshow(xdata.coords['TLAT'])
ax[2].imshow(xdata.coords['TLONG'])
plt.tight_layout()

In [ ]:
# does not work with matplotlib 3.0.0, due to '_hold' bug
# use 'CS' conda environment

In [ ]:
plt.figure(figsize=(7,2));
ax = plt.axes(projection=ccrs.PlateCarree());
xdata.TEMP[0,0,:,:].plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(),x='TLONG', y='TLAT', add_colorbar=False)
ax.coastlines();

In [ ]:
def map_geography(ax):
    """adds land, WGKP ploygon, lat/lon labels"""
    xticks = np.linspace(-180,180,7,endpoint=True)
    yticks = np.linspace(-90,90,7)
    ax.add_feature(cartopy.feature.LAND, zorder=1, facecolor='lightgray')
    gl = ax.gridlines(xlocs=xticks, ylocs=yticks)
    gl.n_steps = 500

In [ ]:
TEMP = shift_field(xdata.TEMP[0,0,:,:].T,shift)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax  = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(central_latitude=-90))
ax.set_position([.02,.02,.96,.96])
ax.pcolormesh(lons.T, lats.T, TEMP.T,
              cmap='rainbow',
              vmin=-1.8, vmax=27,
              transform=ccrs.PlateCarree() )
map_geography(ax)
fig.savefig('../../results/first_map', transparent=True, dpi=300)